In [27]:
import os
import logging
from pathlib import Path
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv

### Meeting Summarization

In [28]:
import os
import logging
from pathlib import Path
from typing import Optional
from openai import OpenAI


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY', ''))

def build_prompt(meeting_transcript: str) -> str:
    try:
        template = {
            'system_role': "당신은 회의 전체 내용을 카테고리별로 요약해주는 요약 AI입니다.",
            'output_format': [
                "다음 회의 날짜와 시간: OOO",
                "아이템: OOO",
                "해야 할 일: OOO",
                "팀원들의 의견: OOO",
                "멘토의 피드백: OOO"
            ]
        }

        prompt = f""" {template['system_role']}
아래 회의 전체 내용을 다음 형식으로 요약하세요:

[출력 예시]
{chr(10).join(template['output_format'])}

[회의 전체 내용]
{meeting_transcript}

[정리된 결과] """
        return prompt

    except Exception as e:
        logging.error(f"Failed to build prompt: {str(e)}")
        raise

def summarize_meeting(transcript: str):
    prompt = build_prompt(transcript)


    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 회의 전체 내용을 카테고리별로 요약해주는 요약 AI입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
    )


    return response.choices[0].message.content.strip()


def get_user_transcript():
    print("회의 내용을 입력하세요 (빈 줄 2번 입력 시 종료):")
    lines = []
    while True:
        line = input()
        if line.strip() == "" and (not lines or lines[-1].strip() == ""):
            break
        lines.append(line)
    return "\n".join(lines).strip()


if __name__ == "__main__":

    if not os.environ.get('OPENAI_API_KEY'):
        api_key = input("OpenAI API 키를 입력하세요: ")
        os.environ['OPENAI_API_KEY'] = api_key
        client = OpenAI(api_key=api_key)


    try:
        meeting_transcript = get_user_transcript()
        summary = summarize_meeting(meeting_transcript)
        print(summary)
    except Exception as e:
        logging.error(f"요약 중 오류 발생: {str(e)}")

회의 내용을 입력하세요 (빈 줄 2번 입력 시 종료):


2025-05-07 17:48:56,887 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


다음 회의 날짜와 시간: 다음 주 화요일 오전 10시
아이템: API 리스크 보험 모델 설계
해야 할 일: 
1. 일일 호출량 기준 1천 건 이상인 API를 '주요 API'로 분류하고, 여기에 장애 빈도와 지속 시간을 결합해서 등급을 나누는 작업 진행
2. 보험료 책정 모델 초안 만들기 (호출량, 장애 확률, 지속 시간 3가지를 주요 변수로 넣어서 회귀 분석)
3. 실시간 API의 예측 오차를 줄이기 위해 구간별로 나눠서 분포 다시 보기
4. 실제 장애 사례를 기반으로 3가지 케이스 만들기 (호출량 급증, 외부 API 장애 전이, 인증 실패 반복)
5. 최근 핀테크 쪽 이슈 기반으로 리스크 요소 추가하기
팀원들의 의견: 
1. 중소 규모 API에서는 관련 데이터가 누락되거나 장애 기록이 1년 이상 안 올라온 것이 많다.
2. 실시간 API는 트래픽이 예측 불가능하여 예측 오차가 크다.
3. 실제 장애 사례를 기반으로 리스크 시나리오를 만드는 것이 좋다.
멘토의 피드백: 리스크 시나리오가 너무 일반적이라고 지난번에 지적함. 최근 핀테크 쪽 이슈 기반으로 리스크 요소를 추가할 것을 제안함.


### Task Alignment

In [ ]:
def build_task_prompt(task_list, team_traits):
    tasks_str = "\n".join(f"{i+1}. {task.strip()}" for i, task in enumerate(task_list.splitlines()))
    return f"""
당신은 팀원 성향을 고려해 업무를 적절히 배분하는 AI입니다.

[팀원 성향]
{team_traits}

[해야 할 일 목록]
{tasks_str}

- 각 업무에 대해 적합한 팀원 1명을 배정하고, 그 이유도 간단히 설명하세요.
- 설명에 포함되는 역량은 해당 인물의 실제 성향과 일치해야 합니다.
- 예: 창의성이 낮은 사람에게 창의성을 이유로 배정하지 마세요.
- 표현은 구체적으로 작성하세요. (예: '중간', '높음' 등 성향 수준 반영)
- 입력된 해야 할 일이 팀원 수보다 적거나 많으면 각 업무에 적합한 팀원 1명이 배정되도록 더 필요한 업무를 추가하거나(해야할 일이 팀원 수보다 적은 경우) 간단한 2개의 업무를 하나로 합치세요(해야할 일이 팀원 수보다 많은 경우).

[출력 예시]
1. 업무명 - 담당자: 이름 (이유: 성향과 업무 특성의 연결)
"""

def assign_tasks(task_list, team_traits):
    prompt = build_task_prompt(task_list, team_traits)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    return response.choices[0].message.content.strip()

print("\n팀원 성향을 입력하세요. 예시:")
print("혜주: 직감형, 창의성 있음, 활동성 중간, 성과 지향적 카리스마형 리더십, 신중한 관계 성향, 실행력 중간\n...")

team_traits_input = []
while True:
    line = input("> ")
    if not line.strip():
        break
    team_traits_input.append(line)
team_traits = "\n".join(team_traits_input)

print("\n해야 할 일 목록을 입력하세요 (한 줄씩 입력, 종료 시 Enter 두 번):")
task_list_input = []
while True:
    line = input("> ")
    if not line.strip():
        break
    task_list_input.append(line)
task_list = "\n".join(task_list_input)

try:
    result = assign_tasks(task_list, team_traits)
    print("\n업무 자동 분배 결과:\n")
    print(result)
except Exception as e:
    logging.error(f"Task assignment 실패: {str(e)}")



팀원 성향을 입력하세요. 예시:
혜주: 직감형, 창의성 있음, 활동성 중간, 성과 지향적 카리스마형 리더십, 신중한 관계 성향, 실행력 중간
...

해야 할 일 목록을 입력하세요 (한 줄씩 입력, 종료 시 Enter 두 번):


2025-05-07 17:50:30,984 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



업무 자동 분배 결과:

1. 일일 호출량 기준 1천 건 이상인 API를 '주요 API'로 분류하고, 여기에 장애 빈도와 지속 시간을 결합해서 등급을 나누는 작업 진행 - 담당자: 윤조 (이유: 분석형으로서 데이터 분석에 강하며, 중간 정도의 실행력을 가지고 있어 이 작업을 체계적으로 진행할 수 있습니다.)

2. 보험료 책정 모델 초안 만들기 (호출량, 장애 확률, 지속 시간 3가지를 주요 변수로 넣어서 회귀 분석) - 담당자: 정아 (이유: 직감형이면서 현실 지향형인 정아는 책임감이 높고 실행력이 높아 회귀 분석을 통해 모델을 체계적으로 작성할 수 있습니다.)

3. 실시간 API의 예측 오차를 줄이기 위해 구간별로 나눠서 분포 다시 보기 - 담당자: 혜주 (이유: 창의성이 높고 직감형인 혜주는 새로운 접근 방식을 통해 예측 오차를 줄이는 데 필요한 창의적 해결책을 제시할 수 있습니다.)

4. 실제 장애 사례를 기반으로 3가지 케이스 만들기 (호출량 급증, 외부 API 장애 전이, 인증 실패 반복) - 담당자: 현서 (이유: 직감형이며 현실 지향적인 현서는 다양한 장애 사례를 현실적으로 분석하고, 신중하게 접근하여 케이스를 만들 수 있습니다.)

5. 최근 핀테크 쪽 이슈 기반으로 리스크 요소 추가하기 - 담당자: 윤희 (이유: 현실적인 편이며 부드러운 리더십을 가진 윤희는 헌신적인 관계형으로 핀테크 이슈에 대한 리스크 요소를 신중하게 고려하여 추가할 수 있습니다.)
